In [ ]:
!pip install git+https://github.com/statsmodels/statsmodels.git --user

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import g_esd_calculations as esd
import generate_synthetic_data as gs
import random

ImportError: cannot import name 'STL' from 'statsmodels.tsa.seasonal' (C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\seasonal.py)

In [ ]:
import dash
import mysql.connector as mysql
import dash_core_components as dcc 
import dash_html_components as html 
import plotly 
import random 
import plotly.graph_objects as go 
from collections import deque
from dash.dependencies import Output, Input
import time
import datetime
import random

In [ ]:
def esd_outlierDetection(alpha,input_series,algo):
    Gcritical=esd.calculate_critical_value(len(input_series),alpha)
    Gstat,max_index=esd.grubbs_stat(input_series,algo)
    return esd.check_G_values(Gstat,Gcritical,input_series,max_index)   

In [ ]:
X = deque(maxlen = 1000) 
X.append(1) 
Y = deque(maxlen = 1000) 
Y.append(1) 

Y_an = deque(maxlen = 1000) 
Y_an.append(1) 

app = dash.Dash(__name__) 

In [ ]:
app.layout = html.Div( 
    [
        dcc.Graph(id = 'live-graph', animate = True),
        dcc.Interval(
            id = 'graph-update',
            interval = 6000,
            n_intervals = 0
        ),
    ]
    
)

In [ ]:
@app.callback(
    Output('live-graph', 'figure'),#live-graph,figure
    [ Input('graph-update', 'n_intervals') ]
)
def update_graph_scatter(n): 

    data=gs.generate_data("temperature")   
       
    
    for i in range(0,len(data)):
        if i in [100]:
            data[i]=data[i]+random.randint(-100,100)
            Y.append(data[i])
        else:
            Y.append(data[i])        
        X.append(X[-1]+1)
        Y_an.append(0)
      
    
    isOutlier,ind,val=esd_outlierDetection(0.01,data)
    time.sleep(5)
    print(len(Y))
    if isOutlier:
        Y_an[-len(data)+ind]=val

  
    data = plotly.graph_objs.Scatter( 
    x=list(X),
    y=list(Y),
    name='Scatter',
    mode= 'lines+markers',

    )
    data_an=go.Scatter(
        x=list(X),
        y=list(Y_an),
        name="anomaly"
    )
    return {'data': [data,data_an],
            'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),yaxis = dict(range = [min(Y),max(Y)]),)}

In [ ]:
if __name__ == '__main__': 
    app.run_server(host = '0.0.0.0',port = '8050')